In [1]:
'''Sequence to sequence example in Keras (character-level).

This script demonstrates how to implement a basic character-level
sequence-to-sequence model. We apply it to translating
short English sentences into short French sentences,
character-by-character. Note that it is fairly unusual to
do character-level machine translation, as word-level
models are more common in this domain.

# Summary of the algorithm

- We start with input sequences from a domain (e.g. English sentences)
    and correspding target sequences from another domain
    (e.g. French sentences).
- An encoder LSTM turns input sequences to 2 state vectors
    (we keep the last LSTM state and discard the outputs).
- A decoder LSTM is trained to turn the target sequences into
    the same sequence but offset by one timestep in the future,
    a training process called "teacher forcing" in this context.
    Is uses as initial state the state vectors from the encoder.
    Effectively, the decoder learns to generate `targets[t+1...]`
    given `targets[...t]`, conditioned on the input sequence.
- In inference mode, when we want to decode unknown input sequences, we:
    - Encode the input sequence into state vectors
    - Start with a target sequence of size 1
        (just the start-of-sequence character)
    - Feed the state vectors and 1-char target sequence
        to the decoder to produce predictions for the next character
    - Sample the next character using these predictions
        (we simply use argmax).
    - Append the sampled character to the target sequence
    - Repeat until we generate the end-of-sequence character or we
        hit the character limit.

# Data download

English to French sentence pairs.
http://www.manythings.org/anki/fra-eng.zip

Lots of neat sentence pairs datasets can be found at:
http://www.manythings.org/anki/

# References

- Sequence to Sequence Learning with Neural Networks
    https://arxiv.org/abs/1409.3215
- Learning Phrase Representations using
    RNN Encoder-Decoder for Statistical Machine Translation
    https://arxiv.org/abs/1406.1078
'''

'Sequence to sequence example in Keras (character-level).\n\nThis script demonstrates how to implement a basic character-level\nsequence-to-sequence model. We apply it to translating\nshort English sentences into short French sentences,\ncharacter-by-character. Note that it is fairly unusual to\ndo character-level machine translation, as word-level\nmodels are more common in this domain.\n\n# Summary of the algorithm\n\n- We start with input sequences from a domain (e.g. English sentences)\n    and correspding target sequences from another domain\n    (e.g. French sentences).\n- An encoder LSTM turns input sequences to 2 state vectors\n    (we keep the last LSTM state and discard the outputs).\n- A decoder LSTM is trained to turn the target sequences into\n    the same sequence but offset by one timestep in the future,\n    a training process called "teacher forcing" in this context.\n    Is uses as initial state the state vectors from the encoder.\n    Effectively, the decoder learns 

In [2]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples =10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'data5000delspace.txt'

E:\anaconda\envs\tensorflow-gpu\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 4999
Number of unique input tokens: 3132
Number of unique output tokens: 3010
Max sequence length for inputs: 53
Max sequence length for outputs: 57


In [4]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

In [5]:
import os.path
from keras.models import load_model
model_name = 's2slstm.h5'

if os.path.isfile(model_name):
    print('file exists')
    #model = load_model(model_name)
    
    # layers[0...4]= 0:encoder_input, 1:decoder_input, 2:encoder_lstm, 3:decoder_lstm, 4:decoder_output
    #encoder_inputs = model.layers[0]
    #encoder = model.layers[2]
    #decoder_lstm = model.layers[3]
    #decoder_dense = model.layers[4]
    
    #encoder_outputs, state_h, state_c = encoder(encoder_inputs)
    # We discard `encoder_outputs` and only keep the states.
    #encoder_states = [state_h, state_c]
    
    encoder_inputs = Input(shape=(None, num_encoder_tokens),name='encoder_input')
    encoder = LSTM(latent_dim, return_state=True, name='encoder_lstm')
    encoder_outputs, state_h, state_c = encoder(encoder_inputs)
    # We discard `encoder_outputs` and only keep the states.
    encoder_states = [state_h, state_c]

    # Set up the decoder, using `encoder_states` as initial state.
    decoder_inputs = Input(shape=(None, num_decoder_tokens),name='decoder_input')
    # We set up our decoder to return full output sequences,
    # and to return internal states as well. We don't use the
    # return states in the training model, but we will use them in inference.
    decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True, name='decoder_lstm')
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                         initial_state=encoder_states)
    decoder_dense = Dense(num_decoder_tokens, activation='softmax', name='decoder_output')
    decoder_outputs = decoder_dense(decoder_outputs)
    
    # Define the model that will turn
    # `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    model.load_weights(model_name)

else:
    print('file not exists')
    # Define an input sequence and process it.
    encoder_inputs = Input(shape=(None, num_encoder_tokens),name='encoder_input')
    encoder = LSTM(latent_dim, return_state=True, name='encoder_lstm')
    encoder_outputs, state_h, state_c = encoder(encoder_inputs)
    # We discard `encoder_outputs` and only keep the states.
    encoder_states = [state_h, state_c]

    # Set up the decoder, using `encoder_states` as initial state.
    decoder_inputs = Input(shape=(None, num_decoder_tokens),name='decoder_input')
    # We set up our decoder to return full output sequences,
    # and to return internal states as well. We don't use the
    # return states in the training model, but we will use them in inference.
    decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True, name='decoder_lstm')
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                         initial_state=encoder_states)
    decoder_dense = Dense(num_decoder_tokens, activation='softmax', name='decoder_output')
    decoder_outputs = decoder_dense(decoder_outputs)
    
    # Define the model that will turn
    # `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    
    # Run training
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
    model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
    # Save model
    model.save_weights(model_name)

file exists


In [6]:
print(decoder_dense)

In [7]:
print(decoder_dense.__dict__)

{'trainable': True, 'activity_regularizer': None, 'supports_masking': True, 'stateful': False, 'name': 'decoder_output', 'bias_constraint': None, 'kernel_constraint': None, '_built': True, '_outbound_nodes': [], '_non_trainable_weights': [], '_initial_weights': None, '_per_input_losses': {}, 'use_bias': True, 'bias_initializer': <keras.initializers.Zeros object at 0x000001FE190FF208>, 'input_spec': InputSpec(min_ndim=2, axes={-1: 256}), '_trainable_weights': [<tf.Variable 'decoder_output/kernel:0' shape=(256, 3010) dtype=float32_ref>, <tf.Variable 'decoder_output/bias:0' shape=(3010,) dtype=float32_ref>], 'bias': <tf.Variable 'decoder_output/bias:0' shape=(3010,) dtype=float32_ref>, 'kernel': <tf.Variable 'decoder_output/kernel:0' shape=(256, 3010) dtype=float32_ref>, 'kernel_regularizer': None, 'activation': <function softmax at 0x000001FBB9E48E18>, '_per_input_updates': {}, '_inbound_nodes': [<keras.engine.base_layer.Node object at 0x000001FE1A2A8CC0>], '_updates': [], 'kernel_initia

In [8]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [9]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [10]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [11]:
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: 中国移动营销行来发展报告alink
Decoded sentence: 去过那么多次，从来没见过

-
Input sentence: 小马也疯狂------地位之争。
Decoded sentence: 就是一天，又一个人，我无聊得

-
Input sentence: 那些年，我们一起偷看过的电视。「暴走漫画」
Decoded sentence: 真不愧是这么走出来的爹········

-
Input sentence: 北京的小纯洁们，周日见。#硬汉摆拍清纯照#
Decoded sentence: 看到的孩子堆：条里是最女人的

-
Input sentence: 要是这一年哭泣的理由不再是难过而是感动会多么好
Decoded sentence: 我已经快感动得哭了。

-
Input sentence: 对于国内动漫画作者引用工笔素材的一些个人意见。
Decoded sentence: 人生的是多少，钱的人出

-
Input sentence: 猫咪保镖最赞了！你们看懂了吗？！（来自：9gag）
Decoded sentence: 要求反雷就是要不cwicne··

-
Input sentence: 莫愁和所有人开了一个玩笑——其实，她是会正常唱歌的……
Decoded sentence: 是，我的天告我小我，老却就下了回床。

-
Input sentence: 你见过皮卡丘喝水的样子吗？
Decoded sentence: 小互你的粉丝肯定大

-
Input sentence: 如果有个人能让你忘掉过去，那TA很可能就是你的未来。
Decoded sentence: 下面的事儿们有必要多么多大王会！

-
Input sentence: 我在北京，24岁，想去马尔代夫，一个人。
Decoded sentence: 这太可以了，哪里没有高呢？

-
Input sentence: 哥你还跳不跳楼了？我们要下班啊！
Decoded sentence: 我想去很多，钱就是我。

-
Input sentence: 龙生龙，凤生凤，是个喵咪它就萌。
Decoded sentence: 老爷子不会长大，你心你呢？

-
Input sentence: 从胚胎期开始的面部特征演变过程
Decoded sentence: 真

In [12]:
num_sample = len(encoder_input_data)
for seq_index in range(num_sample-100, num_sample):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: 工作圆满完成，今天要回台湾啰！北京•再见。
Decoded sentence: 老罗今晚去李志的演出么，有登台献唱的计划么

-
Input sentence: 准备夜战，辞职了，失业了，求保养
Decoded sentence: 哇，我也看过，也会是爱人，我能失去老然和追坐一个。

-
Input sentence: 把答案，变成你喜欢的样子。（via伟大的安妮）
Decoded sentence: 是否小时候也就像单位，我外人其他不想已经失了。

-
Input sentence: 为什么要关灯一小时呢？剩下57分钟做什么？
Decoded sentence: 当然是黑崎一护家的魂，然后是后后一个城后老贝才是多。

-
Input sentence: 一篇日记，多么自律的小童鞋，让人内牛满面呀……
Decoded sentence: 这人说得这么做了，以后会不对啊

-
Input sentence: 今天两家中国公司IPO取消。如果我没弄错的话。
Decoded sentence: 这是要心情。我也不住到它班，这个晚

-
Input sentence: 近年最喜欢的两句诗：他们摧残花朵，以为能阻止春天的到来……
Decoded sentence: 哈哈，这是怎么游的豆腐渣工程啊

-
Input sentence: 狗宝宝的一天，太催泪了！慎入……「转」
Decoded sentence: 不能太不用了一只是老王说这样说

-
Input sentence: #Lens夜话#你平常喜欢收集/珍藏________________
Decoded sentence: 我也不相信作家会不过去的主要还给我吧

-
Input sentence: 难得的早饭就来这一口。
Decoded sentence: 老郑发这个有意义吗？一只要打天比！

-
Input sentence: 朋友们早上好，今天的空气质量。
Decoded sentence: 真心的。现实都是有人吗？这样又有一个人。

-
Input sentence: 主人，救救我，帮我翻下身。
Decoded sentence: 我去天了一个小认。！

-
Input sentence: 我们来#随手拍#标语中国吧
Decoded sentence: 是不是在我这个我，我怕失

In [17]:
fo = open("LSTMresult.txt", "w",encoding="utf8")
for seq_index in range(len(encoder_input_data)):
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)

    fo.write(decoded_sentence)
    
fo.close()

In [ ]:
fo = open("LSTManswer.txt", "w",encoding="utf8")
for seq_index in range(len(encoder_input_data)):
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)

    fo.write(decoded_sentence)
    
fo.close()